# GRT Jacobian Tests

In this Notebook we will test the calculation of the Jacobians.

In [1]:
from gravray import *
from gravray.util import *
from gravray.sampling import *
from gravray.spice import *
from gravray.orbit import *
from gravray.stats import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Initialization

In [144]:
#Preparation
Spice.loadKernels()

## Single ray

In [3]:
#Parameters
body="EARTH"
lon=+61.1*Angle.Deg #deg
lat=+54.8*Angle.Deg #deg
alt=23.3*Const.km #m
#Location
earth=Body(body)
chelyabinsk=Location(earth,lon,lat,alt)
#Ray
A=101.1*Angle.Deg
h=15.9*Angle.Deg
v=-18.6*Const.kms
ray=GrtRay(chelyabinsk,A,h,v)
#Time
tdb=Spice.str2t("02/15/2013 03:20:34 UTC")
#Propagate ray
ray.updateRay(tdb)
print("State Body:",ray.stateBody)
print("State Ecliptic:",ray.stateEcl)
ray.propagateRay()
#Show elements
print("Terminal elements:",Const.transformElements(ray.terminal.elements,[1/Const.au,Angle.Rad]))
print("Geocentric elements:",ray.conics[0])

State Body: [ 1.78729411e+06  3.23767769e+06  5.20762055e+06  1.23526608e+04
 -1.33886204e+04 -2.17876404e+03]
State Ecliptic: [-8.82111395e+05 -1.22185261e+06  6.20687075e+06 -1.53985024e+04
  8.07537430e+03 -5.85361851e+03]
Terminal elements: [  0.73858102   0.54966922   4.04158232 326.57255475 106.86339209
  21.32411541]
Geocentric elements: [6077673.8684861595, 4.229093602118369, 1.803342061748524, 5.880003588638919, 1.9707249679347572, -0.9128197428263376, 398600436233000.0, 414170434.0]


Jacobians:

p(Rimp) = p(Ehel) |dEhel/dehel| x |dehel/dXSoI| x |dXSoI/degeo| x |degeo/dXgeo| x |dXgeo/dRimp|

### dXgeo/dRimp

In [146]:
# Analytical
ray.updateRay(tdb)
ray.calcJacobiansEcliptic()
print("Body to Local:",ray.Jcl)
print("Ecliptic to Local:",ray.Jel)
JEL=np.copy(ray.Jel)
np.linalg.det(JEL)

Body to Local: [[-3.23767769e+06 -2.52801038e+06  2.78579584e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.78729411e+06 -4.57948291e+06  5.04646045e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  3.69000654e+06  8.17144898e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.33886204e+04  1.38595563e+03 -3.66988960e-05 -9.94719861e+03
  -8.97352124e+03 -6.76780178e-01]
 [ 1.23526608e+04  1.72353048e+03  2.02588790e-05 -1.08932417e+04
  -5.90893448e+03  7.26806242e-01]
 [ 0.00000000e+00 -5.75146402e+03  0.00000000e+00  1.01185455e+04
  -1.51828986e+04  1.17137852e-01]]
Ecliptic to Local: [[ 3.58981963e+06  1.26210772e+06 -1.37484659e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-8.17426203e+05  6.12568420e+06 -1.88278200e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 3.49265604e+05  1.36445068e+06  9.72445005e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-9.73737363e+03 -9.00420338e+02  4.06904053e-05  6.803

-7.87694899143873e+21

In [5]:
def Rimp2Xgeo(X):
    lon,lat,alt,A,h,v=X
    earth=Body(body)
    site=Location(earth,lon,lat,alt)
    ray=GrtRay(site,A,h,v)
    tdb=Spice.str2t("02/15/2013 03:20:34 UTC")
    ray.updateRay(tdb)
    return ray.stateEcl

In [6]:
X=[lon,lat,alt,A,h,v]
dX=[1e-3]*6
Rimp2Xgeo(X)

array([-8.82111395e+05, -1.22185261e+06,  6.20687075e+06, -1.53985024e+04,
        8.07537430e+03, -5.85361851e+03])

In [7]:
Jel=Util.computeJacobian(Rimp2Xgeo,X,dX)

In [8]:
for i,comp in enumerate(["lon","lat","alt","A","h","v"]):
    print(f"Jacobian for {comp}:")
    print("\tJ numerical:",Jel[:,i].tolist())
    print("\tJ analytical:",ray.Jel[:,i].tolist())

Jacobian for lon:
	J numerical: [3589819.033004809, -817426.0667483322, 349265.5461872928, -9737.37200333835, -14120.385243092642, 6135.273475984832]
	J analytical: [3589819.631308648, -817426.2029859358, 349265.60439829534, -9737.373626236189, -14120.387596492252, 6135.274498531617]
Jacobian for lat:
	J numerical: [1262107.5144698843, 6125683.171393466, 1364450.4274674691, -900.4317124527006, -4143.64281410144, -4470.970716573902]
	J analytical: [1262107.723430389, 6125684.19526756, 1364450.6760201682, -900.4203382811412, -4143.629317692705, -4471.056702098458]
Jacobian for alt:
	J numerical: [-0.13748457422479987, -0.1882781507447362, 0.9724446572363377, 4.042794898850843e-05, -1.1348674888722599e-05, 1.7721504264045507e-06]
	J analytical: [-0.137484658676238, -0.1882782001630745, 0.9724450051144443, 4.0690405280935404e-05, -9.265480414855458e-06, 3.958906143837841e-06]
Jacobian for A:
	J numerical: [0.0, 0.0, 0.0, 6803.857126707044, 16036.359548899327, 4066.5749698932814]
	J analyti

In [145]:
np.linalg.det(Jel)

-7.876940899535282e+21

### degeo/dXgeo

In [126]:
# degeo/dXgeo
geo_aelements=ray.conics[0]
geo_mu=geo_aelements[-2]
geo_elements=np.array(geo_aelements[:6])
geo=KeplerianOrbit(geo_mu)
geo.setElements(geo_elements,0)
geo.calcJacobians()
print("State:",geo.state)
print("Jck:",geo.Jck)
print("Jkc:",geo.Jkc)
JKC_GEO=np.copy(geo.Jkc)

State: [-8.82111395e+05 -1.22185261e+06  6.20687075e+06 -1.53985024e+04
  8.07537430e+03 -5.85361851e+03]
Jck: [[ 4.68669482e-01 -9.48681250e+05 -2.43524664e+06  1.22185261e+06
  -5.83709488e+06 -1.99156616e+06]
 [ 6.49175412e-01 -2.80155205e+04 -5.70918719e+06 -8.82111395e+05
   2.57298420e+06  1.04442898e+06]
 [-3.29773645e+00  1.68044959e+06 -1.46997488e+06  0.00000000e+00
  -3.23054973e+05 -7.57078072e+05]
 [-4.09064444e-03  1.03994389e+03  2.29664921e+03 -8.07537430e+03
   7.10034509e+03  1.74520507e+02]
 [ 2.14524011e-03 -3.68678573e+02  5.38425966e+03 -1.53985024e+04
   1.31383924e+03  2.41736291e+02]
 [-1.55502603e-03 -2.90933472e+02  1.38631405e+03  0.00000000e+00
  -1.68656255e+04 -1.22799256e+03]]
Jkc: [[-2.39847920e-02 -3.32224261e-02  1.68766104e-01 -2.73705943e+02
   1.43538500e+02 -1.04047142e+02]
 [-2.77316470e-07 -8.18444442e-08  7.77291689e-07 -4.97371345e-04
   2.43518216e-04 -1.21817851e-04]
 [-5.87924982e-08 -1.37833011e-07 -3.54886005e-08  1.12614910e-06
   2.6401

In [128]:
def egeo2Xgeo(X,mu=1):
    a,e,i,W,w,M=X
    q=a*(1-e)
    orbit=KeplerianOrbit(mu)
    orbit.setElements([q,e,i,W,w,M],0)
    return orbit.state

In [129]:
a=geo_elements[0]/(1-geo_elements[1])
X=[a]+geo_elements[1:].tolist()
dX=[1e-6]*6
args=dict(mu=geo_mu)
egeo2Xgeo(X,**args)

array([-8.82111395e+05, -1.22185261e+06,  6.20687075e+06, -1.53985024e+04,
        8.07537430e+03, -5.85361851e+03])

In [130]:
Jck=Util.computeJacobian(egeo2Xgeo,X,dX,N=6,**args)
Jck

array([[ 4.68804501e-01, -9.48681250e+05, -2.43524664e+06,
         1.22185261e+06, -5.83709488e+06, -1.99156616e+06],
       [ 6.49248250e-01, -2.80155207e+04, -5.70918719e+06,
        -8.82111395e+05,  2.57298420e+06,  1.04442898e+06],
       [-3.29734758e+00,  1.68044959e+06, -1.46997488e+06,
         0.00000000e+00, -3.23054972e+05, -7.57078072e+05],
       [-4.08908818e-03,  1.03994389e+03,  2.29664921e+03,
        -8.07537430e+03,  7.10034509e+03,  1.74520507e+02],
       [ 2.14504325e-03, -3.68678573e+02,  5.38425966e+03,
        -1.53985024e+04,  1.31383924e+03,  2.41736291e+02],
       [-1.55478119e-03, -2.90933471e+02,  1.38631405e+03,
         0.00000000e+00, -1.68656255e+04, -1.22799256e+03]])

In [131]:
Jany=geo.Jck
Jnum=Jck
for i,comp in enumerate(["a","e","i","W","w","M"]):
    print(f"Jacobian for {comp}:")
    print("\tJ numerical:",Jnum[:,i].tolist())
    print("\tJ analytical:",Jany[:,i].tolist())

Jacobian for a:
	J numerical: [0.4688045009970665, 0.6492482498288155, -3.2973475754261017, -0.004089088179171085, 0.0021450432541314512, -0.0015547811926808208]
	J analytical: [0.46866948168866973, 0.6491754118019047, -3.297736449054449, -0.004090644444246504, 0.0021452401074557015, -0.0015550260258418197]
Jacobian for e:
	J numerical: [-948681.2498653308, -28015.520656481385, 1680449.5924152434, 1039.9438942840789, -368.6785730678821, -290.93347120578983]
	J analytical: [-948681.2498987756, -28015.520495045937, 1680449.5918342883, 1039.943894881852, -368.6785728447588, -290.93347167540077]
Jacobian for i:
	J numerical: [-2435246.642213315, -5709187.18772009, -1469974.8754501343, 2296.649211530166, 5384.259658057999, 1386.3140511602978]
	J analytical: [-2435246.6425018953, -5709187.188169952, -1469974.8753572349, 2296.649211928477, 5384.259658804754, 1386.3140513666594]
Jacobian for W:
	J numerical: [1221852.6074429974, -882111.39512714, 0.0, -8075.374301370175, -15398.50243489127, 0.

### degeo/dRimp

Composed

In [132]:
def Rimp2egeo(X):
    lon,lat,alt,A,h,v=X
    earth=Body(body)
    site=Location(earth,lon,lat,alt)
    ray=GrtRay(site,A,h,v)
    tdb=Spice.str2t("02/15/2013 03:20:34 UTC")
    ray.updateRay(tdb)
    ray.propagateRay()
    aelements=ray.conics[0]
    mu=geo_aelements[-2]
    elements=np.array(aelements[:6])
    celements=np.copy(elements)
    celements[0]=celements[0]/(1-celements[1])
    return celements

In [133]:
lon=+61.1*Angle.Deg #deg
lat=+54.8*Angle.Deg #deg
alt=23.3*Const.km #m
A=101.1*Angle.Deg
h=15.9*Angle.Deg
v=-18.6*Const.kms

X=[lon,lat,alt,A,h,v]
dX=[1e-8]*6
Rimp2egeo(X)

array([-1.88216095e+06,  4.22909360e+00,  1.80334206e+00,  5.88000359e+00,
        1.97072497e+00, -9.12819743e-01])

In [134]:
Jkl=Util.computeJacobian(Rimp2egeo,X,dX,N=6)
Jkl

array([[ 1.92783773e+01,  1.13370898e+05,  1.51339918e-01,
         1.64635596e+04,  2.39036279e+04, -3.26265581e+02],
       [ 8.88622509e-05,  1.92720773e-01,  7.10542736e-07,
         2.57209365e-02, -1.13170553e+00, -5.63993297e-04],
       [-1.54783630e-01, -9.81921389e-01, -3.33066907e-08,
         5.27647925e-02, -1.33260070e-04,  1.11022302e-08],
       [ 8.30752001e-01, -4.88479923e-02, -4.44089210e-08,
        -1.04181468e+00,  2.63149502e-03, -1.33226763e-07],
       [-3.76497922e-01,  1.73202863e-01,  2.22044605e-08,
        -2.37598641e-01,  1.26281927e+00,  9.03721542e-06],
       [-2.94542168e-05, -3.56827567e-02, -3.44169138e-07,
        -1.56852864e-02, -3.15471586e+00,  1.65212288e-04]])

In [135]:
np.linalg.det(Jkl)

0.0003408722631640826

In [137]:
np.linalg.det(JKC_GEO)*np.linalg.det(JEL)

0.0003506354890956949

### dXSoI/degeo

In [138]:
from gravray.orbit import *

In [143]:
soi_aelements=ray.conics[1]
soi_mu=soi_aelements[-2]
soi_elements=np.array(soi_aelements[:6])
soi=KeplerianOrbit(soi_mu)
soi.setElements(soi_elements,0)
print("SoI State = ",soi.state)
print("SoI celements = ",soi.celements)
soi.calcJacobians()
print("Jck = ",soi.Jck)
print("Jkc = ",soi.Jkc)
JCK_SOI=np.copy(soi.Jck)

SoI State =  [ 1.33514266e+09 -6.31599250e+08  2.41171932e+08 -1.30172897e+04
  6.13849935e+03 -2.27591024e+03]
SoI celements =  [-1.88216095e+06  4.22909360e+00  1.80334206e+00  5.88000359e+00
  1.97072497e+00 -7.90185673e+02]
Jck =  [[-7.09366889e+02 -2.16541035e+07 -9.46230655e+07  6.31599250e+08
  -3.61418489e+08 -1.68359187e+06]
 [ 3.35571329e+02 -1.25027151e+07 -2.21834119e+08  1.33514266e+09
  -2.15614856e+08  7.93923150e+05]
 [-1.28135658e+02  8.44323435e+07 -5.71168137e+07  0.00000000e+00
   1.43616654e+09 -2.94354976e+05]
 [-3.45807028e-03  1.98623638e+02  8.92946380e+02 -6.13849935e+03
   3.45172355e+03 -2.05352775e-02]
 [ 1.63070522e-03  1.22629700e+02  2.09342164e+03 -1.30172897e+04
   2.13099490e+03  9.71436708e-03]
 [-6.04600325e-04 -8.05329044e+02  5.39004436e+02  0.00000000e+00
  -1.39948289e+04 -3.70936583e-03]]
Jkc =  [[ 2.82221480e-06 -1.33506989e-06  5.09787470e-07 -2.31380263e+02
   1.09110854e+02 -4.04539442e+01]
 [-1.22443110e-07 -7.55961205e-08  4.96451734e-07 

In [21]:
def egeo2Xgeo(X,mu=1):
    a,e,i,W,w,M=X
    q=a*(1-e)
    orbit=KeplerianOrbit(mu)
    orbit.setElements([q,e,i,W,w,M],0)
    return orbit.state

In [22]:
X=np.copy(soi.celements)
dX=[1e-3]*6
args=dict(mu=soi_mu)
egeo2Xgeo(X,**args)

array([ 1.33514266e+09, -6.31599250e+08,  2.41171932e+08, -1.30172897e+04,
        6.13849935e+03, -2.27591024e+03])

In [23]:
soi.celements

array([-1.88216095e+06,  4.22909360e+00,  1.80334206e+00,  5.88000359e+00,
        1.97072497e+00, -7.90185673e+02])

In [24]:
X

array([-1.88216095e+06,  4.22909360e+00,  1.80334206e+00,  5.88000359e+00,
        1.97072497e+00, -7.90185673e+02])

In [25]:
Jsk=Util.computeJacobian(egeo2Xgeo,X,dX,N=6,**args)
Jsk

array([[-7.09366918e+02, -2.16541036e+07, -9.46230497e+07,
         6.31599145e+08, -3.61418428e+08, -1.68359187e+06],
       [ 3.35571408e+02, -1.25027163e+07, -2.21834082e+08,
         1.33514243e+09, -2.15614820e+08,  7.93923150e+05],
       [-1.28135681e+02,  8.44323486e+07, -5.71168042e+07,
        -2.98023224e-05,  1.43616630e+09, -2.94354976e+05],
       [-3.45806984e-03,  1.98623639e+02,  8.92946231e+02,
        -6.13849832e+03,  3.45172297e+03, -2.05352781e-02],
       [ 1.63070536e-03,  1.22629713e+02,  2.09342130e+03,
        -1.30172875e+04,  2.13099454e+03,  9.71436702e-03],
       [-6.04600245e-04, -8.05329094e+02,  5.39004346e+02,
         4.54747351e-10, -1.39948266e+04, -3.70936596e-03]])

In [26]:
Jany=soi.Jck
Jnum=Jsk
for i,comp in enumerate(["a","e","i","W","w","M"]):
    print(f"Jacobian for {comp}:")
    print("\tJ numerical:",Jnum[:,i].tolist())
    print("\tJ analytical:",Jany[:,i].tolist())

Jacobian for a:
	J numerical: [-709.3669176101685, 335.57140827178955, -128.13568115234375, -0.0034580698411446065, 0.001630705355637474, -0.0006046002454240806]
	J analytical: [-709.3668891596213, 335.57132923629536, -128.13565838617563, -0.0034580702828349444, 0.0016307052171732688, -0.0006046003252553279]
Jacobian for e:
	J numerical: [-21654103.624343872, -12502716.339230537, 84432348.58614206, 198.62363917036419, 122.62971255086086, -805.3290937423299]
	J analytical: [-21654103.483045492, -12502715.0840171, 84432343.47683838, 198.62363788326863, 122.62970032729076, -805.329044135356]
Jacobian for i:
	J numerical: [-94623049.68380928, -221834081.82615042, -57116804.20285463, 892.9462311407406, 2093.4212960564764, 539.0043463835354]
	J analytical: [-94623065.45423755, -221834118.79861742, -57116813.722043455, 892.9463799666311, 2093.421644959422, 539.0044362174394]
Jacobian for W:
	J numerical: [631599145.1780796, 1335142432.5506687, -2.9802322387695312e-05, -6138.498324919055, -130

In [27]:
np.linalg.det(Jsk)

-2.2464766684822025e+25

### dehel/dXSoI

In [28]:
#dehel/dXSoI
hel_aelements=ray.conics[2]
hel_mu=hel_aelements[-2]
hel_elements=np.array(hel_aelements[:6])
hel=KeplerianOrbit(hel_mu)
hel.setElements(hel_elements,0)
hel.calcJacobians()
print(hel.celements)
JKC_SOI=np.copy(hel.Jkc)
hel.Jkc

[2.45353314e+11 5.49669222e-01 7.05389185e-02 5.69976633e+00
 1.86511804e+00 3.72176024e-01]


array([[-4.63083553e+00,  3.20779936e+00,  8.91263190e-03,
        -2.76497054e+07, -1.67426303e+07, -2.06349992e+06],
       [-7.04262342e-12,  1.78749803e-13, -2.63580119e-13,
        -4.94181576e-05, -1.13930475e-05, -2.59533869e-06],
       [-1.24372606e-13, -1.88424499e-13,  3.19534184e-12,
         1.19042720e-06,  1.80349723e-06, -3.05840810e-05],
       [-3.72952926e-12, -5.65023687e-12,  9.58178919e-11,
        -3.79239828e-07, -5.74548344e-07,  9.74331031e-06],
       [-8.02363391e-13,  1.98722941e-11, -9.49161607e-11,
        -4.82852550e-05, -1.05403021e-04, -1.78624700e-05],
       [ 9.11928673e-12, -8.65837341e-12, -1.55620352e-13,
         8.27731981e-05,  5.93535376e-05,  6.72177813e-06]])

In [29]:
def XSoI2ehel(X,mu=1):
    orbit=KeplerianOrbit(mu)
    orbit.setState(X,0)
    return orbit.celements

In [30]:
X=np.copy(hel.state)
dX=1e-6*np.abs(X)
args=dict(mu=hel_mu)
XSoI2ehel(X,**args)

array([2.45353314e+11, 5.49669222e-01, 7.05389185e-02, 5.69976633e+00,
       1.86511804e+00, 3.72176024e-01])

In [31]:
Jkc=Util.computeJacobian(XSoI2ehel,X,dX,N=6,**args)
Jkc

array([[-4.63083553e+00,  3.20779936e+00,  8.91271548e-03,
        -2.76497054e+07, -1.67426303e+07, -2.06349994e+06],
       [-7.04262342e-12,  1.78749803e-13, -2.63580066e-13,
        -4.94181576e-05, -1.13930475e-05, -2.59533870e-06],
       [-1.24372605e-13, -1.88424499e-13,  3.19534180e-12,
         1.19042720e-06,  1.80349723e-06, -3.05840810e-05],
       [-3.72952926e-12, -5.65023687e-12,  9.58178918e-11,
        -3.79239823e-07, -5.74548352e-07,  9.74331030e-06],
       [-8.02363392e-13,  1.98722941e-11, -9.49161611e-11,
        -4.82852550e-05, -1.05403021e-04, -1.78624699e-05],
       [ 9.11928673e-12, -8.65837341e-12, -1.55620488e-13,
         8.27731981e-05,  5.93535376e-05,  6.72177811e-06]])

In [32]:
Jany=hel.Jkc
Jnum=Jkc
for i,comp in enumerate(["a","e","i","W","w","M"]):
    print(f"Jacobian for {comp}:")
    print("\tJ numerical:",Jnum[:,i].tolist())
    print("\tJ analytical:",Jany[:,i].tolist())

Jacobian for a:
	J numerical: [-4.630835530267583, -7.042623419604562e-12, -1.2437260547885655e-13, -3.72952925736321e-12, -8.023633916550607e-13, 9.119286726785078e-12]
	J analytical: [-4.630835530205663, -7.0426234195356236e-12, -1.2437260550898445e-13, -3.72952925693729e-12, -8.023633914578409e-13, 9.119286726793469e-12]
Jacobian for e:
	J numerical: [3.2077993555770665, 1.7874980297533247e-13, -1.8842449902643194e-13, -5.650236871866014e-12, 1.9872294094374604e-11, -8.658373409125542e-12]
	J analytical: [3.2077993553568054, 1.7874980272966839e-13, -1.8842449892304537e-13, -5.650236871547289e-12, 1.9872294094192657e-11, -8.658373408239716e-12]
Jacobian for i:
	J numerical: [0.008912715475943028, -2.6358006596284397e-13, 3.1953418008887808e-12, 9.581789180032997e-11, -9.491616113662525e-11, -1.5562048782077091e-13]
	J analytical: [0.008912631902059872, -2.6358011911928413e-13, 3.1953418380279232e-12, 9.581789190691436e-11, -9.491616073728448e-11, -1.5562035160798993e-13]
Jacobian for

## dehel/degeo

Combined

In [33]:
np.linalg.det(JCK_SOI)*np.linalg.det(JKC_SOI)

-1.5314356135953121e-09

In [35]:
def egeo2ehel(X,mu_geo=1,mu_hel=1,body=None,tpos=0):
    ageo,egeo,igeo,Wgeo,wgeo,Mgeo=X
    geo=KeplerianOrbit(mu_geo)
    qgeo=ageo*(1-egeo)
    geo.setElements([qgeo,egeo,igeo,Wgeo,wgeo,Mgeo],0)
    body.updateBody(tpos)
    state=geo.state+body.stateHelio
    hel=KeplerianOrbit(mu_hel)
    hel.setState(state,0)
    return hel.celements

In [36]:
a=soi_elements[0]/(1-soi_elements[1])
X=np.array([a]+soi_elements[1:].tolist())
dX=np.abs(X*1e-6)
args=dict(mu_geo=geo_mu,mu_hel=hel_mu,tpos=hel_aelements[-1],body=earth)
egeo2ehel(X,**args)

array([2.45353314e+11, 5.49669222e-01, 7.05389185e-02, 5.69976633e+00,
       1.86511804e+00, 3.72176024e-01])

In [37]:
Jee=Util.computeJacobian(egeo2ehel,X,dX,N=6,**args)
Jee

array([[ 7.39201876e+04, -5.82230951e+09, -6.11252491e+10,
         3.89029404e+11, -1.01244450e+11,  1.07533644e+07],
       [ 1.58971464e-07, -8.99462652e-03, -6.87353198e-02,
         4.47450469e-01, -1.56406775e-01,  1.29901698e-05],
       [ 1.69310954e-08,  2.53626972e-02, -1.17754278e-02,
        -3.11142096e-02,  4.40645872e-01, -7.74244966e-07],
       [-1.70444312e-08,  2.49178622e-04, -1.56223261e-04,
        -9.24144550e-05,  1.28729436e-03, -2.64452568e-05],
       [ 2.52920909e-08, -1.63760452e-02, -2.72308183e-01,
         1.69448624e+00, -2.81608627e-01,  4.51008573e-05],
       [-2.02865884e-07,  1.82036210e-02,  2.02853792e-01,
        -1.28652584e+00,  3.16469646e-01, -2.33295549e-05]])

In [38]:
np.linalg.det(Jee)

-1.531428660447717e-09

## degeoSoIdegeo

In [87]:
def egeo2egeoSoI(X,mu=1,body=None):
    a,e,i,W,w,Mo=X
    q=a*(1-e)
    #Find the M at SoI
    n=np.sqrt(mu/np.abs(a)**3)
    fd=np.arccos((q*(1+e)/body.rhill-1)/e)
    Hd=2*np.arctanh(np.sqrt((e-1)/(e+1))*np.tan(fd/2))
    Md=e*np.sinh(Hd)-Hd
    
    asoi=a
    esoi=e
    isoi=i
    Wsoi=W
    wsoi=w
    return np.array([asoi,esoi,isoi,Wsoi,wsoi,-Md])

In [88]:
geo.elements

array([ 6.07767387e+06,  4.22909360e+00,  1.80334206e+00,  5.88000359e+00,
        1.97072497e+00, -9.12819743e-01])

In [89]:
X=np.copy(geo.elements)
X[0]=X[0]/(1-X[1])
dX=np.abs(X*1e-6)
args=dict(mu=geo_mu,body=earth)
egeo2egeoSoI(X,**args)

array([-1.88216095e+06,  4.22909360e+00,  1.80334206e+00,  5.88000359e+00,
        1.97072497e+00, -7.90185673e+02])

In [78]:
Jsg=Util.computeJacobian(egeo2egeoSoI,X,dX,N=6,**args)
Jsg

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [-4.21930055e-04, -2.31147739e-01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [79]:
np.linalg.det(Jsg)

0.0

## dEheldehel

In [114]:
#dehel/dXSoI
hel_aelements=ray.conics[2]
hel_mu=hel_aelements[-2]
hel_elements=np.array(hel_aelements[:6])
hel=KeplerianOrbit(hel_mu)
hel.setElements(hel_elements,0)
print(hel.elements)

[1.10490149e+11 5.49669222e-01 7.05389185e-02 5.69976633e+00
 1.86511804e+00 3.72176024e-01]


In [115]:
#dEhel/dehel
helE=Const.transformElements(hel.elements,[1/Const.au,1.0])
hel.elements=helE
hel.calcUelements()
hel.uelements
hel.calcJacobiansMap()
print(hel.uelements)
JEE=np.copy(hel.JEe)
print(hel.JEe)

[ 1.03860642  0.19933431 -3.77361143  2.27927487 -0.86237799 -2.76520495]
[[ 5.17922462  0.          0.          0.          0.          0.        ]
 [ 0.          4.03986591  0.          0.          0.          0.        ]
 [ 0.          0.         14.50219233  0.          0.          0.        ]
 [ 0.          0.          0.         -0.21545809  0.          0.        ]
 [ 0.          0.          0.          0.          1.31956676  0.        ]
 [ 0.          0.          0.          0.          0.          3.04798752]]


In [116]:
np.linalg.det(JEE)

-262.94971623862756

## Summarizing

In [117]:
def Rimp2Ehel(X):
    lon,lat,alt,A,h,v=X
    #Definition
    earth=Body(body)
    site=Location(earth,lon,lat,alt)
    ray=GrtRay(site,A,h,v)
    #Propagate
    tdb=Spice.str2t("02/15/2013 03:20:34 UTC")
    ray.updateRay(tdb)
    ray.propagateRay()
    #Final elements
    hel=ray.terminal
    #"""
    helE=Const.transformElements(hel.elements,[1/Const.au,1.0])
    hel.elements=helE
    hel.calcUelements()
    #"""
    return hel.uelements

In [118]:
X=np.array([lon,lat,alt,A,h,v])
dX=np.abs(X*1e-6)
Rimp2Ehel(X)

array([ 1.03860642,  0.19933431, -3.77361143,  2.27927487, -0.86237799,
       -2.76520495])

In [119]:
Jhl=Util.computeJacobian(Rimp2Ehel,X,dX,N=6)
Jhl

array([[ 2.03243905e+00,  1.79192479e-01, -4.71071060e-08,
        -2.32352388e+00, -2.99120053e-01,  8.19588930e-05],
       [ 1.78255203e+00,  1.37095118e-01,  7.60461669e-08,
        -1.73863576e+00, -7.37570557e-01, -1.84522237e-04],
       [-2.75437649e+00,  1.39611114e+00,  2.56569587e-07,
        -1.04363003e+00,  7.65849224e+00, -2.32089230e-04],
       [-1.01512145e-03,  7.23436896e-04,  2.28229453e-10,
        -4.11566922e-04,  2.67711408e-03,  3.53841210e-09],
       [ 1.18633848e+00,  1.00888442e-01, -8.29546017e-09,
        -1.30637516e+00, -2.53770588e-01,  5.68755308e-06],
       [-3.48245782e+00, -2.83019775e-01, -5.33288435e-08,
         3.63647013e+00,  1.06045068e+00,  1.52616364e-04]])

In [120]:
np.linalg.det(Jhl)

7.2131940667282455e-22

In [121]:
np.linalg.det(JKC_SOI)*np.linalg.det(JCK_SOI)*np.linalg.det(JKC)*np.linalg.det(JEL)*np.linalg.det(JEE)

1.4119760147125552e-10

In [122]:
np.linalg.det(JKC_SOI)*np.linalg.det(JEL)*np.linalg.det(JEE)

1.4119760147126358e-10

In [123]:
np.linalg.det(JCK_SOI)

-2.2464779625568697e+25

In [124]:
np.linalg.det(JKC)

-4.451412462830372e-26

In [125]:
1/np.linalg.det(JCK_SOI)

-4.451412462830625e-26